In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug

set_debug(True)

chat = ChatOpenAI(
    max_tokens=1000,
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

chat.predict("엄청 맛있는 마이야르 숙성 스테이크는 어떻게 만들어?")